# Part 2 - Ray-Tracing Intermediate Report
Use this notebook to draft the narrative for your next intermediate report. Replace the prompts in each section with your own text.

## 1. Input Wave Conditions
The values below reuse the EVA outputs from `Part_1_Mean_EVA_Waves/index_project_EVA_1.ipynb` (ResourceCode hindcast at the Bretagne Sud 1 site, 47.3236 deg N / -3.5522 deg E). Heights and directions come straight from the BM and POT fits so the ray-tracing report references the same forcing.
### 1-year storm setup
| Parameter | Block Maxima (BM) | Peak-over-Threshold (POT) | Notes |
| --- | --- | --- | --- |
| Source / dataset | EVA fit to 27 annual maxima in the notebook | EVA fit to 137 declustered peaks above threshold | Both rely on the ResourceCode hindcast node collocated with Bretagne Sud 1. |
| Offshore location | Bretagne Sud 1 (47.3236 deg N, -3.5522 deg E) | Same | Deep-water point west of the ray-tracing domain edge. |
| Significant wave height (H<sub>s</sub>) | 6.96 m (95% CI 6.42-7.96 m) for the 1.01-year return | 8.51 m (95% CI 8.22-8.76 m) for the 1.01-year return | Table reproduced from Part 1 (BM - POT = -1.55 m at this horizon). |
| Wave period (T<sub>p</sub>) | 13.9 s median of BM extremes (10%-90%: 12.8-18.3 s); T ~= 12.7sqrt(H/g) cross-check -> 10.7 s | 13.9 s median of POT extremes (10%-90%: 12.2-18.0 s); T ~= 12.7sqrt(H/g) cross-check -> 11.8 s | Periods fall within the mid-teens expected for energetic Atlantic swell. |
| Wave direction | Mean 255.9 deg (coming from WSW), circular std 11.6 deg | Mean 260.4 deg (WSW), circular std 12.6 deg | Both directions point toward the coastline, matching buoy climatology. |
| Reasonableness check | Moderate-depth swell propagating shoreward; BM amplitude slightly lower because it samples one event per year. | Larger H<sub>s</sub> reflects the fatter POT tail but is still within recorded extremes. | Either forcing option launches deep-water rays that refract toward shore with physical H-T pairs. |

### 50-year storm setup
| Parameter | Block Maxima (BM) | Peak-over-Threshold (POT) | Notes |
| --- | --- | --- | --- |
| Source / dataset | Same EVA workflow focused on the BM tail | Same EVA workflow focused on the POT tail | Extreme values cross over so both are documented for the ray model. |
| Offshore location | Bretagne Sud 1 (47.3236 deg N, -3.5522 deg E) | Same | Identical launch section for both storm severities. |
| Significant wave height (H<sub>s</sub>) | 12.84 m (95% CI 11.45-13.64 m) at 50-year return | 11.82 m (95% CI 10.51-13.08 m) at 50-year return | BM exceeds POT by +1.02 m at this horizon; both lie within each other's CI. |
| Wave period (T<sub>p</sub>) | Use the upper tail of BM quantiles (15-18 s) for propagation; T ~= 12.7sqrt(H/g) gives 14.5 s as a consistency check. | POT quantiles suggest 14-18 s, and the T ~= 12.7sqrt(H/g) check with H = 11.82 m gives 13.9 s. | Longer periods keep the waves in deep water at the ray-launch boundary. |
| Wave direction | Retain BM mean 255.9 deg (WSW) for the severe case | Retain POT mean 260.4 deg (WSW) | Severe storms share the same propagation sector; only H<sub>s</sub> and T differ. |
| Reasonableness check | H<sub>s</sub> > 12 m with T ~15 s is energetic but in line with hindcast extremes and remains within deep/intermediate water at the launch section. | POT offers a slightly lower H<sub>s</sub>, giving a conservative alternative forcing. | Both satisfy the expected storm climatology (Atlantic swell impinging from the west-southwest). |

### Additional considerations
- Retain the notebook CSV exports (e.g., `summary_bm_pot.csv`) with the full return-level table in case reviewers need to trace the numbers.
- Document any subsequent tweaks (e.g., if you refine T using dispersion in the ray code) so the intermediate report states which combination feeds the simulations.


## 2. Ray-Tracing Runs and Quality Checks
Describe the simulations you executed for each case.
- **Coverage:** _Do the traced rays span the region of interest? Do they originate in deep water?_
- **Input alignment:** _Do ray headings match your intended incident directions?_
- **Validation tests (if needed):** _If the default runs showed no/limited refraction, document any alternative parameters explored to confirm the model can capture refraction._

### 3.1 Dataset

The bathymetric data for the ray-tracing simulations was obtained via the GMRT Map Tool, covering the "Bretagne Sud" region of interest. The dataset provides a high-resolution grid of the local topography and bathymetry required to resolve wave refraction around the complex coastline and islands.

**Grid Specifications:**
* **Source File:** `GMRTv4_4_0_20251107topo.asc`
* **Coordinate System:** Geographic Lat/Lon (WGS84)
* **Grid Dimensions:** 1225 columns $\times$ 703 rows
* **Spatial Resolution:** $0.001099^\circ$ per grid cell. At the domain latitude ($\approx 47.1^\circ$N), this corresponds to a physical resolution of approximately **83 m (East-West) $\times$ 122 m (North-South)**.
* **Domain Extent:**
    * **Longitude:** $-4.04^\circ$ W to $-2.69^\circ$ W
    * **Latitude:** $47.14^\circ$ N to $47.91^\circ$ N

This resolution ($\approx 100$ m order of magnitude) allows for detailed modeling of wave interactions with the seabed, specifically focusing on the shoaling and refraction processes occurring between the offshore boundary and the shoreline.

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from ocean_wave_tracing import Wave_tracing
import cmocean
from scipy.ndimage import gaussian_filter

### Load Bathymetry

In [ ]:
# bathymetry file path
base_path = Path.cwd() 
fname = base_path / 'GMRTv4_4_0_20251107topo.asc' # bathymetry file

# load bathymetry
with fname.open() as fid:
    ncols = int(fid.readline().split()[1])
    nrows = int(fid.readline().split()[1])
    lonll = float(fid.readline().split()[1])
    latll = float(fid.readline().split()[1])
    dd = float(fid.readline().split()[1])
dx = dd*6378.e3*np.pi/180*np.cos(np.pi/180*latll)
dy = dd*6378.e3*np.pi/180
bathy = -np.flipud(np.loadtxt(fname,skiprows=6))

# apply gaussian smoothing to bathymetry
#sigma_smooth = 2.0
#bathy = gaussian_filter(bathy, sigma=sigma_smooth)
#print(f"Bathymetry smoothed with sigma={sigma_smooth}")

# --- MASK VERY SHALLOW WATER ---
#min_depth_limit = 5.0  # Set anything shallower than 5m to Land
#bathy[bathy < min_depth_limit] = np.nan
#print(f"Depths < {min_depth_limit}m masked as land.")

# paths for created figs

fig_dir = base_path / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

print(f"Bathymetry loaded. Figures will be saved to: {fig_dir}")

### 1 Year Return Period

In [ ]:
# define parameters for 1 year return period
period_one_year= 13.9 # median wave period (s) for 1 year return period
mean_wave_direction = ((259.9 + 260.4)/2) - 180 # wave direction (deg)
ang_one_year   = mean_wave_direction
nr_case1    = 100  # number of wave rays for Case 1(less rays so its easier to see the main direction of the rays)
nr_case2    = 1000  # number of wave rays for Case 2(more rays so the energy flux resolution is better)
simtime = 3600*2.5 # propagate 2.5 hour
nt    = 20000 # number of time steps, try to increase to limit edge cases

In [ ]:
# ray tracing for Case 1 (less rays so its easier to see the main direction of the rays)
ang = 90-ang_one_year # convert
wt_case1 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case1,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case1.set_initial_condition(wave_period=period_one_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case1.solve()

In [ ]:
# plotting wave rays --> Case 1
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = cmocean.cm.deep.reversed()
cmap.set_bad(color='gray')
pc=ax.pcolormesh(wt_case1.x,wt_case1.y,-wt_case1.d,shading='auto',cmap=cmap)
fig.colorbar(pc)
for ray_id in range(wt_case1.nb_wave_rays):
    ax.plot(wt_case1.ray_x[ray_id,:],wt_case1.ray_y[ray_id,:],'-k')
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim((-np.max(wt_case1.d),0))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Wave rays')
fig.savefig(fig_dir / "wave_rays_1_year.png", dpi=300, bbox_inches='tight')
plt.show()

#### Energy Flux

In [ ]:
wt_case2 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case2,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case2.set_initial_condition(wave_period=period_one_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case2.solve()

In [ ]:
# wave energy flux density estimation --> Case 2
from scipy.interpolate import RegularGridInterpolator
xx,yy,hm=wt_case2.ray_density(x_increment=20,y_increment=20)
interp = RegularGridInterpolator(points=(xx[0,:],yy[:,0]),values=hm.transpose(),bounds_error=False,fill_value=None)
xg,yg=np.meshgrid(wt_case2.x,wt_case2.y,indexing='ij')
e = interp((xg,yg))
e = e.transpose()
e[np.isnan(wt_case2.d)]=np.nan
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    escale = e[round(nrows/2),0]
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    escale = e[0,round(ncols/2)]
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    escale = e[round(nrows/2),-1]
else:
    escale = e[-1,round(ncols/2)]

rel_e = e/escale

In [ ]:
vmax = np.percentile(rel_e[~np.isnan(rel_e)], 98)
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = plt.cm.get_cmap("jet")
cmap.set_bad(color='gray')
pc = ax.pcolormesh(wt_case2.x,wt_case2.y,rel_e,shading='auto',cmap=cmap, vmax=vmax)
fig.colorbar(pc)
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim(vmax=vmax)
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Relative wave energy flux density')
fig.savefig(fig_dir / "relative_wave_energy_flux_density_1_year.png", dpi=300, bbox_inches='tight')
plt.show()

### 50 Year Retrurn Period 

In [ ]:
# define parameters for 50 year return period
period_BM = 18.32
period_POT = 17.99
mean_period = (period_BM + period_POT) / 2
period_50_year= mean_period # upper end of the POT and BM quantiles (90th percentile)
mean_wave_direction = ((259.9 + 260.4)/2) - 180 # wave direction (deg)
ang_50_year   = mean_wave_direction
nr_case1    = 100  # number of wave rays for Case 1(less rays so its easier to see the main direction of the rays)
nr_case2    = 1000  # number of wave rays for Case 2(more rays so the energy flux resolution is better)
simtime = 3600*2.5 # propagate 2.5 hour
nt    = 15000 # number of time steps, try to increase to limit edge cases
print(f"successuflly initialized the wave period {period_50_year:.2f} s and wave direction {ang_50_year:.2f} deg")

In [ ]:
# ray tracing for Case 1 (less rays so its easier to see the main direction of the rays)
ang = 90-ang_50_year # convert
wt_case1 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case1,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case1.set_initial_condition(wave_period=period_50_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case1.solve()

In [ ]:
# plotting wave rays --> Case 1
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = cmocean.cm.deep.reversed()
cmap.set_bad(color='gray')
pc=ax.pcolormesh(wt_case1.x,wt_case1.y,-wt_case1.d,shading='auto',cmap=cmap)
fig.colorbar(pc)
for ray_id in range(wt_case1.nb_wave_rays):
    ax.plot(wt_case1.ray_x[ray_id,:],wt_case1.ray_y[ray_id,:],'-k')
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim((-np.max(wt_case1.d),0))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Wave rays')
fig.savefig(fig_dir / "wave_rays_50_year.png", dpi=300, bbox_inches='tight')
plt.show()

#### Energy Flux

In [ ]:
wt_case2 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case2,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case2.set_initial_condition(wave_period=period_50_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case2.solve()

In [ ]:
# wave energy flux density estimation --> Case 2
from scipy.interpolate import RegularGridInterpolator
xx,yy,hm=wt_case2.ray_density(x_increment=20,y_increment=20)
interp = RegularGridInterpolator(points=(xx[0,:],yy[:,0]),values=hm.transpose(),bounds_error=False,fill_value=None)
xg,yg=np.meshgrid(wt_case2.x,wt_case2.y,indexing='ij')
e = interp((xg,yg))
e = e.transpose()
e[np.isnan(wt_case2.d)]=np.nan
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    escale = e[round(nrows/2),0]
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    escale = e[0,round(ncols/2)]
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    escale = e[round(nrows/2),-1]
else:
    escale = e[-1,round(ncols/2)]

rel_e = e/escale

In [ ]:
vmax = np.percentile(rel_e[~np.isnan(rel_e)], 98)
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = plt.cm.get_cmap("jet")
cmap.set_bad(color='gray')
pc = ax.pcolormesh(wt_case2.x,wt_case2.y,rel_e,shading='auto',cmap=cmap, vmax=vmax)
fig.colorbar(pc)
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim(vmax=vmax)
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Relative wave energy flux density')
fig.savefig(fig_dir / "relative_wave_energy_flux_density_50_year.png", dpi=300, bbox_inches='tight')
plt.show()

## 3. Model Configuration Recap
The ray-tracing simulations were performed using the `ocean_wave_tracing` module with specific numerical settings chosen to maximize resolution while maintaining stability on the raw bathymetry grid.

* **Number of rays:**
    * **Visualization (Case 1):** 100 rays were used to generate clear, uncluttered plots of the wave paths.
    * **Energy Flux (Case 2):** 1000 rays were used to estimate the ray density and relative energy flux. This higher number ensures sufficient spatial resolution to identify energy focusing hotspots.
* **Time step and total propagation time:**
    * **Total Duration ($T$):** 9,000 seconds (2.5 hours). This duration allowed waves to propagate fully from the offshore boundary ($x=0$) to the coastline without stopping prematurely.
    * **Time Steps ($nt$):** 20,000 steps. This extremely high temporal resolution ($dt \approx 0.45$ s) was strictly necessary to navigate the raw, high-frequency changes in bathymetry without numerical divergence.
* **Bathymetry/topography source:**
    * Topography was obtained from the GMRT Map Tool (File: `GMRTv4_4_0_20251107topo.asc`).
    * **Grid Size:** 1225 columns $\times$ 703 rows, with a spatial resolution of approx. 83m $\times$ 122m.
* **Pre-processing:**
    * **Raw Bathymetry:** No spatial smoothing (Gaussian filter) or artificial depth masking was applied. This decision was made to strictly preserve the physical topology of the island channels and ridges, ensuring that no potential wave focusing features were artificially flattened, even at the cost of increased computational demand.

**Conclusion:** By avoiding smoothing filters, the simulation preserves the full complexity of the underwater terrain. Stability was achieved primarily through the refined time-stepping ($nt=20,000$), and the use of 1000 rays ensures that the resulting energy flux map captures fine-scale refraction effects around the archipelago.

### Bathymetry Plot

In [ ]:
### Plot Bathymetry Contours ###
# -----------------------------
x_vec = np.linspace(0, dx * (ncols - 1), ncols)
y_vec = np.linspace(0, dy * (nrows - 1), nrows)
X, Y = np.meshgrid(x_vec, y_vec)

# Target Site
target_lat = 47.32
target_lon = -3.55
x_idx = (target_lon - lonll) / dd
y_idx = (target_lat - latll) / dd
poi_x = x_idx * dx
poi_y = y_idx * dy
poi_depth = bathy[int(y_idx), int(x_idx)]

# 2. Create Plot
# 2. Create Plot (Large & Legible)
# ---------------------------------------
# Set global minimum font size to 15 as requested
plt.rcParams.update({'font.size': 20})

# Increase figure size to accommodate larger text
fig, ax = plt.subplots(figsize=(15, 10))
ax.set_aspect('equal')

# A. Background (Elevation)
elevation = -bathy 
cmap = plt.get_cmap('terrain') 
cmap.set_bad('gray')
pc = ax.pcolormesh(X, Y, elevation, cmap=cmap, shading='auto', vmin=-120, vmax=50)

# Colorbar (Large Labels)
cb = fig.colorbar(pc, label='Depth (m)', pad=0.02)
cb.ax.tick_params(labelsize=20) 
pc.set_clim(-120, 0)
cb.set_label('Depth (m)', fontsize=20)



# B. Contour Lines (Thicker & Smoother)
levels = [-100, -80, -60, -50, -40, -30, -20, -10, 0]
CS = ax.contour(X, Y, elevation, levels=levels, colors='white', linewidths=1.2, alpha=0.8, fontsize=20)
ax.clabel(CS, inline=True, fontsize=20, fmt='%1.0f m')


# C. Mark Site (Red Dot)
# 'ro' = Red Dot. Markersize 14 makes it stand out.
ax.plot(poi_x, poi_y, 'ro', markersize=14, markeredgecolor='black', label='Site (47.32N, 3.55W)')

# D. Annotation Box (Large Text)
ax.text(poi_x + 3500, poi_y, f" Depth: {poi_depth:.1f}m", 
        color='white', fontweight='bold', fontsize=18,
        bbox=dict(facecolor='red', alpha=0.7, edgecolor='none', boxstyle='round,pad=0.4'))

# E. Titles and Labels (Publication Sized)
ax.set_title('Bathymetry & Topography Contours', fontsize=24, fontweight='bold', pad=20)
ax.set_xlabel('x (m)', fontsize=20)
ax.set_ylabel('y (m)', fontsize=20)

# Ticks (Readable)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlim((0, dx*(ncols-1)))
ax.set_ylim((0, dy*(nrows-1)))

# Legend
ax.legend(loc='upper right', fontsize=18, framealpha=0.95, facecolor='white')

# Save
fig.savefig(fig_dir / "bathymetry_contours.png", dpi=300, bbox_inches='tight')
plt.show()

## 4. Wave Energy Flux Interpretation

The spatial distribution of wave energy is determined by the interaction between the incoming wave characteristics (Period, Direction) and the local underwater topography. Therefore, the analysis begins with the bathymetric context before evaluating the resulting energy flux densities.

### 4.1 Bathymetric Control
The underlying bathymetry acts as an optical lens for waves, causing convergence (focusing) over ridges and divergence (calming) over canyons or depressions.

![Bathymetry and Topography with Contours](figures/bathymetry_contours.png)
*Figure 4.1: Bathymetric map of the study domain with depth contours. The red dot indicates the site of interest ($47.32^\circ$N, $-3.55^\circ$W).*

**Bathymetric Analysis:**
[REPLACE THIS TEXT WITH YOUR DESCRIPTION. Suggestion: Describe the general slope (is it steep?), mention the depth at your specific site (approx 25-30m), and identify if there are any ridges/underwater hills to the South-West of your site that might direct waves toward it.]
### 4.2 Visualization of Energy Flux Density
The refraction simulations yield the Relative Wave Energy Flux Density ($E/E_0$), visualizing how energy redistributes for the two storm scenarios.

![Relative Wave Energy Flux Density (1-Year Return Period)](figures/relative_wave_energy_flux_density_1_year.png)
*Figure 4.2: Relative wave energy flux density for the 1-year storm scenario ($T_p \approx 13.9$ s).*

![Relative Wave Energy Flux Density (50-Year Return Period)](figures/relative_wave_energy_flux_density_50_year.png)
*Figure 4.3: Relative wave energy flux density for the 50-year storm scenario ($T_p \approx 18.3$ s).*

### 4.3 Scenario 1: 1-Year Return Period Analysis
**Global Refraction Pattern:**
For the 1-year storm ($T_p \approx 13.9$ s), the wave field exhibits moderate refractive features. The focusing bands (areas of red/yellow) are somewhat diffuse. This indicates that while the bathymetry steers the waves, the shorter wavelength allows the waves to propagate further onto the shelf before experiencing sharp directional changes.

**Site-Specific Results ($47.32^\circ$ N, $-3.55^\circ$ W):**
* **Flux Density:** The site lies within a zone of moderate energy concentration. The model estimates a relative energy factor between **1.0 and 1.2**.
* **Implication:** The local wave height is expected to be slightly higher than the offshore significant wave height ($H_s \approx 6.96$ m), but the amplification factor is not severe.

### 4.4 Scenario 2: 50-Year Return Period Analysis
**Global Refraction Pattern:**
The 50-year scenario ($T_p \approx 18.3$ s) demonstrates a significantly intensified refraction pattern. Due to the longer period (and thus longer wavelength), these waves "feel" the bottom at greater depths ($h \approx \lambda/2$). Consequently, the refractive process initiates further offshore. The resulting caustics (high-energy bands) are sharper, more elongated, and spatially distinct compared to the 1-year case. Shadow zones behind the islands also become more pronounced due to the enhanced interaction with the seabed.

**Site-Specific Results ($47.32^\circ$ N, $-3.55^\circ$ W):**
* **Flux Density:** The site lies on the periphery of a primary caustic generated by the bathymetric features to the southwest. The relative energy factor increases to approximately **1.2 – 1.4**.
* **Implication:** For the design storm case ($H_s \approx 12.84$ m), the local conditions are exacerbated by the bathymetry. The waves are not only larger due to the storm intensity but are further amplified by approximately 30% (in terms of energy density) due to the refractive focusing of the longer-period swell.

## 5. Wave Height Estimates at Points of Interest
Pick at least one point (e.g., high energy region) and estimate the wave height using your model output or modified code.
- _Location (coordinates or description):_
- _Depth at that point:_
- _Estimated wave height:_
- _Applicable period:_
- _Comments on the validity of linear wave theory (cite depth-to-wavelength ratio, steepness, etc.)._

## 6. Summary Table for Floating Body Prep
Fill in the table with the case and location(s) you intend to use moving forward.
| Case | Location / Notes | Water Depth (m) | Wave Height (m) | Wave Period (s) | Wave Direction (deg / cardinal) |
| --- | --- | --- | --- | --- | --- |
| 1-year | _e.g., near offshore buoy or control point_ |  |  |  |  |
| 50-year | _e.g., same location for comparison_ |  |  |  |  |

## 7. Bonus Cross-Check (Optional)
If you had time to compare against another dataset (e.g., ResourceCode station), describe:
- _Station/data source:_
- _Observed vs. modeled differences:_
- _Potential reasons for agreement/mismatch._